### Beta estimation method using the Fujitsu stock data and the Nikkei225 Market indices data
#### Compiler: Mashele Given Phazamisa

In [1]:
## Import the neccessary libraries
import pandas as pd
import numpy as np

### Reading the stock data: Fujitsu 

In [2]:
# Now lets read the Fujitsu data dataset
data = pd.read_csv('FJTSY.csv')
## Lets select the date and Adjusted Prices only
data = data[['Date','Adj Close']]
### Lets rename the columns
data.rename(columns={"Date":'date','Adj Close':'price_t'},inplace=True)
# Lets calcuate the returns 
data['stock_returns'] = (data['price_t']/data['price_t'].shift(1))-1
# Lets now select the date and the  stock returns columns only
data = data[['date','stock_returns']]
# Lets drop the first row with missing values the dataset
data =data.drop(index=0,axis=0)

In [3]:
# Let display the five rows of the dataset
data.head()

,date,stock_returns
1,2023-02-02,0.005180
2,2023-02-03,-0.028856
3,2023-02-06,-0.019455
4,2023-02-07,-0.015512
5,2023-02-08,-0.001099


### Reading the stock market indices dataset: NIKKEI225

In [4]:
# Now lets read the Nikkei 225 data dataset
df = pd.read_csv('^N225.csv')
## Lets select the date and Adjusted Prices only
df = df[['Date','Adj Close']]
### Lets rename the columns
df.rename(columns={"Date":'date','Adj Close':'price_t'},inplace=True)
# Lets calcuate the returns 
df['returns_market'] = (df['price_t']/df['price_t'].shift(1))-1
# Lets select the date and returns columns
df = df[['date','returns_market']]
## Lets drop the first row of the dataset
df = df.drop(index=0,axis=0)

In [5]:
# Lets print the first five rows of the market dataset
df.head()

,date,returns_market
1,2023-02-02,0.002017
2,2023-02-03,0.003920
3,2023-02-06,0.006695
4,2023-02-07,-0.000295
5,2023-02-08,-0.002854


### Concatenation of the fujitsu and Nikkei dataset

In [6]:
# Lets create a new dataframe by combining the stock and market dataframes
new_data = pd.concat([data,df],axis=1,ignore_index=False)

In [7]:
new_data.head()

,date,stock_returns,date,returns_market
1,2023-02-02,0.005180,2023-02-02,0.002017
2,2023-02-03,-0.028856,2023-02-03,0.003920
3,2023-02-06,-0.019455,2023-02-06,0.006695
4,2023-02-07,-0.015512,2023-02-07,-0.000295
5,2023-02-08,-0.001099,2023-02-08,-0.002854


In [8]:
# Lets drop the last 20 rows of the dataset
new_data.tail(20)

,date,stock_returns,date,returns_market
210,2023-12-01,0.010908,2023-12-07,-0.017568
211,2023-12-04,-0.004873,2023-12-08,-0.016752
212,2023-12-05,0.003498,2023-12-11,0.014979
213,2023-12-06,0.029976,2023-12-12,0.001583
214,2023-12-07,0.003723,2023-12-13,0.002517
215,2023-12-08,0.006069,2023-12-14,-0.007292
216,2023-12-11,0.001005,2023-12-15,0.008698
217,2023-12-12,0.021426,2023-12-18,-0.006417
218,2023-12-13,-0.017044,2023-12-19,0.014054
219,2023-12-14,-0.005669,2023-12-20,0.013744


In [9]:
## Lets drop the rows with missing values of the dataset
new_data = new_data.drop(index=[227,228,229],axis=0)

In [10]:
# Lets set the index as date
new_data.set_index('date',inplace=True)

In [11]:
# Print the first five rows of the new dataframe
new_data.head()

,stock_returns,returns_market
date,,
"(2023-02-02, 2023-02-02)",0.005180,0.002017
"(2023-02-03, 2023-02-03)",-0.028856,0.003920
"(2023-02-06, 2023-02-06)",-0.019455,0.006695
"(2023-02-07, 2023-02-07)",-0.015512,-0.000295
"(2023-02-08, 2023-02-08)",-0.001099,-0.002854


In [12]:
## Lets make a copy of the new dataframe
fujitsu_Nikkei_data= new_data.copy()

### Question 1 :  Create a function to calculate the beta of the stock given the dataframe information

In [13]:
## Lets define the Beta function:
def beta_function(data,stock_return_column,market_return_column):
    """
    Calculate the beta using the stock return: Fujitsu
    and market returns: Nikkei225
    
    """
    # Calculate the stock returns deviations:
    stock_dev = data[stock_return_column] -  data[stock_return_column].mean()
    # Calculate the market return deviations:
    market_dev = data[market_return_column] -  data[market_return_column].mean()
    # Now lets compute the sum of the products of the deviations
    sum_value = np.sum(stock_dev*market_dev)
    # now lets calculate the number of observation minus 1.
    divisor = len(data[stock_return_column])-1 
    ## Now lets compute the beta value:
    covariance = sum_value/divisor
    # now lets calculate the market variance
    market_variance = np.var(data[market_return_column],ddof=1)
    # Calculate the beta
    beta =covariance/market_variance
    return beta

In [15]:
## now lets compute the beta:
beta_function(fujitsu_Nikkei_data,stock_return_column='stock_returns' ,market_return_column='returns_market')

0.3577427339018736

### Question 2 :  Calculate the beta using the covariance and variance built in functions

In [16]:
## lets compute the covariance matrix between the stock and the market
cov_matrix =np.cov(fujitsu_Nikkei_data['stock_returns'],fujitsu_Nikkei_data['returns_market'])
print(cov_matrix)

[[2.89080329e-04 3.65940340e-05]
 [3.65940340e-05 1.02291481e-04]]


In [17]:
## Now lets compute the covariance between the stock and the market in the first row and column 2
stock_market_covariance = cov_matrix[0][1]
print(stock_market_covariance)

3.659403398220608e-05


In [18]:
## calcuate the market variance
market_var = np.var(fujitsu_Nikkei_data['returns_market'],ddof=1)
print(market_var)

0.00010229148076070655


In [19]:
### Alternative approach of calculating the variance of the market
market_variance = cov_matrix[1][1]
print(market_variance)

0.00010229148076070658


In [20]:
### now lets compute the beta
beta = stock_market_covariance/market_var
print(beta)

0.3577427339018737


### Question 3 : Estimate the Beta using the Scipy function. i.e linregress

In [21]:
from scipy.stats import linregress

In [22]:
linregress(y=fujitsu_Nikkei_data['stock_returns'],x=fujitsu_Nikkei_data['returns_market'])

LinregressResult(slope=0.35774273390187367, intercept=-7.667819038207171e-05, rvalue=0.2128047335579765, pvalue=0.0012892564395732792, stderr=0.10974941323836232, intercept_stderr=0.0011123797127281954)

In [23]:
### Now lets compute the beta:
beta = linregress(y=fujitsu_Nikkei_data['stock_returns'],x=fujitsu_Nikkei_data['returns_market'])[0]
print(beta)

0.35774273390187367


### Question4: Comment on the beta estimates

 #### Observation:
* The beta values are approximately the same , with a slight difference at the last digits.
* The Fujitsu stock is less risky than the market(Nikkei 225) with the beta values of less than 1